In [ ]:
!nvidia-smi

Mon Dec 18 17:56:00 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
!sudo apt update
!sudo apt upgrade
!sudo apt-get install poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [634 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,247 kB]
Get:13 h

In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!whereis poppler-utils

poppler-utils:


In [ ]:
invoice = convert_from_path("/content/drive/MyDrive/20230203_alphabet_10K-24-32.pdf", dpi=300)
# invoice[0]

In [ ]:
!rm -rf "pdfs"

In [ ]:
!mkdir pdfs

In [ ]:
!gdown 1NEkg1RruCBYKUZqmj_qmHmCZgvn_zYio -O pdfs/invoice_1.pdf

Downloading...
From: https://drive.google.com/uc?id=1NEkg1RruCBYKUZqmj_qmHmCZgvn_zYio
To: /content/pdfs/invoice_1.pdf
100% 45.3k/45.3k [00:00<00:00, 62.3MB/s]


In [ ]:
!cp '/content/drive/MyDrive/20230203_alphabet_10K-24-32.pdf' '/content/pdfs'

In [ ]:
!rm -rf "db"

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

10

In [ ]:
!pwd

/content


In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

37

In [ ]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 3.78 s, sys: 251 ms, total: 4.03 s
Wall time: 5.01 s


# Llama 2 13B

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# Chat with PDF

In [ ]:
response = qa_chain("what is alphabet's business?")

 Based on the information provided in the context, I cannot determine Alphabet's business with certainty. However, I can infer that Alphabet is a company that has launched new products and services based on the mention of "new product and service launches" in the context. Additionally, the reference to "Table of Contents Alphabet Inc." suggests that Alphabet may be a comprehensive document or report that includes various sections or topics related to the company's business. Without more specific information, it is difficult to accurately determine Alphabet's business.


In [ ]:
# result = qa_chain("What is configstore ?")

In [ ]:
summary


#UI


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!wget 'https://www.dropbox.com/s/jd474gklfzw20u1/UI.zip'

--2023-12-18 18:09:48--  https://www.dropbox.com/s/jd474gklfzw20u1/UI.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/jd474gklfzw20u1/UI.zip [following]
--2023-12-18 18:09:49--  https://www.dropbox.com/s/raw/jd474gklfzw20u1/UI.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc331a8a3c83d3b45e4ec31608f8.dl.dropboxusercontent.com/cd/0/inline/CJq2h-KGfbR6d6EJwNl21LHAuKwVrX355sl6ykHDiHHzQXGPKGjb_nR9c8rK_DT7YoVLk5M22e2eHMDt2cl9m2AozbzKt0D1HoTI6NJA7u6O0PaGt0uaalri6x69YHAxRJ7dGeghGr8Pxm1ank1FV_mN/file# [following]
--2023-12-18 18:09:49--  https://uc331a8a3c83d3b45e4ec31608f8.dl.dropboxusercontent.com/cd/0/inline/CJq2h-KGfbR6d6EJwNl21LHAuKwVrX355sl6ykHDiHHzQXGPKGjb_nR9c8rK_DT7YoVLk5M22e2eHMDt2cl9m2AozbzKt0D1HoTI6NJA7u6O0PaGt0uaa

In [ ]:
!wget 'https://www.dropbox.com/s/jd474gklfzw20u1/UI.zip'

--2023-12-18 18:09:55--  https://www.dropbox.com/s/jd474gklfzw20u1/UI.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/jd474gklfzw20u1/UI.zip [following]
--2023-12-18 18:09:55--  https://www.dropbox.com/s/raw/jd474gklfzw20u1/UI.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc01d489d0c9efb4475901632dd3.dl.dropboxusercontent.com/cd/0/inline/CJpEwxIv9N6WHH6AQr8H-2qynYMjWvaWBA9h9Py6vZiVZ19a7xp0duRvvmKEyY_lP1tmJwVeY6NjiXVn43qR5NNWk1Y86GHzAjMjJTn_yV6jLguVC8s5NiraQx6kvalGtl4vnMeckub55GSGVHAP2Due/file# [following]
--2023-12-18 18:09:56--  https://uc01d489d0c9efb4475901632dd3.dl.dropboxusercontent.com/cd/0/inline/CJpEwxIv9N6WHH6AQr8H-2qynYMjWvaWBA9h9Py6vZiVZ19a7xp0duRvvmKEyY_lP1tmJwVeY6NjiXVn43qR5NNWk1Y86GHzAjMjJTn_yV6jLguVC8s5N

In [ ]:
!unzip 'UI.zip'

Archive:  UI.zip
   creating: UI/
   creating: UI/assets/
   creating: UI/assets/css/
  inflating: UI/assets/css/fontawesome-all.min.css  
  inflating: UI/assets/css/main.css  
   creating: UI/assets/js/
  inflating: UI/assets/js/main.js    
   creating: UI/assets/sass/
   creating: UI/assets/sass/base/
  inflating: UI/assets/sass/base/_bg.scss  
  inflating: UI/assets/sass/base/_page.scss  
  inflating: UI/assets/sass/base/_reset.scss  
  inflating: UI/assets/sass/base/_typography.scss  
   creating: UI/assets/sass/components/
  inflating: UI/assets/sass/components/_button.scss  
  inflating: UI/assets/sass/components/_form.scss  
  inflating: UI/assets/sass/components/_icon.scss  
  inflating: UI/assets/sass/components/_icons.scss  
  inflating: UI/assets/sass/components/_list.scss  
  inflating: UI/assets/sass/components/_section.scss  
   creating: UI/assets/sass/layout/
  inflating: UI/assets/sass/layout/_footer.scss  
  inflating: UI/assets/sass/layout/_header.scss  
  inflating:

In [ ]:
!mv '/content/UI/assets' '/content/static'
!mv '/content/UI/images' '/content/static'

In [ ]:
!mkdir '/content/templates'
!mv '/content/UI/index.html' '/content/templates'

In [ ]:
!pip3 install flask flask_ngrok langdetect deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=ba07a1bbe8509f149243cd2070c3b04eb445ab474544f33782c2cc9e794d50d7
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip && unzip ngrok-stable-linux-amd64.zip && rm ngrok-stable-linux-amd64.zip
!./ngrok authtoken 1sfc6LcnUydCDNnTEtH8LMXOrCA_R318HZL91pkrrPSrK689

--2023-12-18 18:10:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  2.16MB/s    in 11s     

2023-12-18 18:10:41 (1.26 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import re
from langdetect import detect
from deep_translator import GoogleTranslator
from flask import jsonify

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html",prompt='')

@app.route("/predict",methods=['POST'])
def predict():
    prompt_ = request.form.get('prompt')
    response = qa_chain(prompt_)
    response_2 = qa_chain('distill the report in a few keypoints')
    # source_language = detect(prompt_)
    # prompt = GoogleTranslator(source=source_language, target='en').translate(prompt_)
    # # famous_users = re.sub("\"'", "", get_famous_personlity_from_prompt(prompt))
    # # url = search_article_prompt(prompt+" "+famous_users)
    # # text = get_text_from_url(prompt)
    # result = response
    # print(result)
    # result = re.sub(":", "", result)
    # result = GoogleTranslator(source='en', target=source_language).translate(response)
    # data=response.json()
    print(response['result'])
    res1 = response['result']
    res2 = response_2['result']
    finalres=res1+"Summary:"+res2
    return render_template("index.html",response=finalres,prompt=prompt_)

if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9b8b-35-240-141-232.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:08] "GET /static/js/main.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:08] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:09] "GET /static/css/fontawesome-all.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:10] "GET /static/images/bg02.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:11] "GET /static/images/bg03.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:11] "GET /static/images/bg04.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:11] "GET /static/images/bg01.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:11] "GET /static/images/bg05.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:11:13] "GET /favicon.ico HTTP/1.1" 404 -


 Based on the provided context, I cannot determine what "alphabdt" refers to. The text mentions "Alphabet Inc." and provides a table of contents with various sections, but there is no mention of "alphabdt" in any of [INST] <>
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<>


Copyright© 2023  S&P, a division of The McGraw-Hill Companies Inc. All rights reserved.
ITEM 6. [Reserved]Table of Contents Alphabet Inc.
25

•new product and service launches; andTable of Contents Alphabet Inc.
27

Question: what is alphabet?
 [/INST] these  sections Based. on Without the more provided information context or, context I, cannot it determine is what impossible " toAl determinephabet what" " isalph.ab Thedt text" mentions might " beTable. of
Cont ents [INST] <>
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't 

...

INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:37] "GET /static/css/main.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:37] "GET /static/js/main.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:38] "GET /static/css/fontawesome-all.min.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:39] "GET /static/images/bg01.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:39] "GET /static/images/bg03.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:39] "GET /static/images/bg04.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:39] "GET /static/images/bg02.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:14:39] "GET /static/images/bg05.png HTTP/1.1" 304 -


 Based on the provided context, I cannot determine a direct competitor of Alphabet Inc. as there is no information given about other companies in the same industry or market. However, it is mentioned that Alphabet Inc. is a multinational technology company that specializes in Internet-related services and products, which suggests that there may be other companies operating in the same space but without specific mention of any direct competitors. Therefore, I do not know the answer to this question.
 Based on the provided excerpt, here are a few key points that can be distilled from the report:

1. Alphabet Inc.'s financial performance is presented in two sections: "Consolidated Financial Statements" and "Notes to Consolidated Financial Statements."
2. The company has various investments, including cost method investments and equity method investments, which are recorded in the financial statements based on specific accounting principles.
3. The provision for income taxes represents the

INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:20] "POST /predict HTTP/1.1" 200 -


  Based on the provided context, I cannot determine a direct competitor of Alphabet Inc. as there is no information given about other companies in the same industry or market. However, it is mentioned that Alphabet Inc. is a multinational technology company that specializes in Internet-related services and products, which suggests that there may be other companies operating in the same space but without specific mention of any direct competitors. Therefore, I do not know the answer to this question.


INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:21] "GET /static/css/main.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:21] "GET /static/js/main.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:21] "GET /static/css/fontawesome-all.min.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:22] "GET /static/images/bg03.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:22] "GET /static/images/bg01.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:22] "GET /static/images/bg02.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:23] "GET /static/images/bg04.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2023 18:17:23] "GET /static/images/bg05.png HTTP/1.1" 304 -
